In [1]:
import pandas as pd
import numpy as np
import gc
import pickle as pkl

from functionUtils import *
from tqdm import tqdm_notebook, tnrange
from sklearn.preprocessing import LabelEncoder

from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Dense, Flatten, Concatenate, Dot, Reshape, Add, Subtract
from keras import objectives
from keras import backend as K
from keras import regularizers 
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2

import os
import numpy as np
import tensorflow as tf
import random as rn

# The below is necessary in Python 3.2.3 onwards to
# have reproducible behavior for certain hash-based operations.
# See these references for further details:
# https://docs.python.org/3.4/using/cmdline.html#envvar-PYTHONHASHSEED
# https://github.com/fchollet/keras/issues/2280#issuecomment-306959926
def init_seeds(seed):
    os.environ['PYTHONHASHSEED'] = '0'
    np.random.seed(seed)
    rn.seed(seed)
    session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

    from keras import backend as K
    tf.set_random_seed(seed)
    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    K.set_session(sess)
    return sess

/home/sjtu123/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df_historical_trans = pd.read_csv('./datasets/historical_transactions.csv',dtype={'purchase_date':np.str},low_memory=True)
#C2:区域；C3分期等级；C1为-1城市编码对应Y
df_historical_trans['category_2'].fillna(6,inplace=True)
df_historical_trans['category_3'].fillna('B',inplace=True)
df_historical_trans['merchant_id'].fillna(df_historical_trans['merchant_id'].value_counts().index[0],inplace=True)
df_historical_trans['installments'].replace(-1,1,inplace=True)
df_historical_trans['installments'].replace(999,0,inplace=True)

df_historical_trans.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [3]:
def label_encoding(df,encodCols):
    for col in encodCols:
        lbl = LabelEncoder()
        lbl.fit(list(df[col].values.astype('str')))
        df[col] = lbl.transform(list(df[col].values.astype('str')))
    return df

for df_feature in [df_historical_trans]:  

    df_feature['category_1'] = df_feature['category_1'].map({'Y': 1, 'N': 0}).astype(np.int16)
    df_feature['category_3'] = df_feature['category_3'].map({'A': 0, 'B': 1, 'C':2}).astype(np.int16)

    df_feature = label_encoding(df_feature,['merchant_category_id'])

In [12]:
features = ['category_1', 'category_3', 'merchant_category_id']
f_size  = [df_historical_trans[f].nunique() + 1 for f in features]
f_size

[3, 4, 328]

In [13]:
X = df_historical_trans.groupby(features)['card_id'].count()
X = X.unstack().fillna(0)
X.head()

merchant_category_id      0      1       2         3      4     5       6    \
category_1 category_3                                                         
0          0             34.0  394.0  3727.0  185847.0   18.0   3.0  3710.0   
           1              8.0  221.0  2996.0  130303.0  146.0  34.0  2213.0   
           2              4.0    5.0   180.0    1768.0   33.0   1.0   372.0   
1          0              1.0    0.0     0.0       0.0    0.0   0.0     0.0   
           1           1157.0    0.0    24.0      15.0    0.0   0.0     7.0   

merchant_category_id       7       8      9     ...     317       318   319  \
category_1 category_3                           ...                           
0          0           32968.0  7888.0  890.0   ...    53.0  617375.0  60.0   
           1           26989.0  4151.0  582.0   ...    70.0  326334.0  21.0   
           2            9602.0   191.0  211.0   ...     1.0   48236.0   8.0   
1          0              57.0     5.0    0.0   ...     0.0      91.0   0.0   
           1           10429.0    95.0    0.0   ...     0.0  273259.0  12.0   

merchant_category_id   320       321     322    323     324     325     326  
category_1 category_3                                                        
0          0           1.0  199150.0  2470.0  310.0  3395.0  8659.0  2760.0  
           1           0.0  104654.0  4958.0  196.0  1901.0  4341.0  3863.0  
           2           0.0  100032.0   120.0  171.0  1120.0   793.0   476.0  
1          0           0.0       2.0     0.0    0.0     0.0     0.0   133.0  
           1           0.0    2806.0     6.0    0.0    10.0    13.0  8203.0  

[5 rows x 327 columns]

In [14]:
X = X.stack().astype('float32')
X = np.log1p(X).reset_index()
X.columns=features + ['num']
X.head()

,category_1,category_3,merchant_category_id,num
0,0,0,0,3.555348
1,0,0,1,5.978886
2,0,0,2,8.223627
3,0,0,3,12.132685
4,0,0,4,2.944439


In [15]:
X_train = [X[f].values for f in features]
y_train = (X[['num']].values).astype('float32')
(X.num > 0).mean()

0.7319062181447502

In [16]:

k_latent = 2
embedding_reg = 0.0002
kernel_reg = 0.1

def get_embed(x_input, x_size, k_latent):
    if x_size > 0: #category
        embed = Embedding(x_size, k_latent, input_length=1, 
                          embeddings_regularizer=l2(embedding_reg))(x_input)
        embed = Flatten()(embed)
    else:
        embed = Dense(k_latent, kernel_regularizer=l2(embedding_reg))(x_input)
    return embed

def build_model_1(X, f_size):
    dim_input = len(f_size)
    
    input_x = [Input(shape=(1,)) for i in range(dim_input)] 
     
    biases = [get_embed(x, size, 1) for (x, size) in zip(input_x, f_size)]
    
    factors = [get_embed(x, size, k_latent) for (x, size) in zip(input_x, f_size)]
    
    s = Add()(factors)
    
    diffs = [Subtract()([s, x]) for x in factors]
    
    dots = [Dot(axes=1)([d, x]) for d,x in zip(diffs, factors)]
    
    x = Concatenate()(biases + dots)
    x = BatchNormalization()(x)
    output = Dense(1, activation='relu', kernel_regularizer=l2(kernel_reg))(x)
    model = Model(inputs=input_x, outputs=[output])
    opt = Adam(clipnorm=0.5)
    model.compile(optimizer=opt, loss='mean_squared_error')
    output_f = factors + biases
    model_features = Model(inputs=input_x, outputs=output_f)
    return model, model_features

In [17]:
model, model_features = build_model_1(X_train, f_size)
# model.summary()

In [18]:
n_epochs = 300
P = 17
try:
    del sess
except:
    pass
sess = init_seeds(0)

batch_size = 2**P
print(batch_size)
model, model_features = build_model_1(X_train, f_size)
earlystopper = EarlyStopping(patience=0, verbose=10)

model.fit(X_train,  y_train, 
          epochs=n_epochs, batch_size=batch_size, verbose=1, shuffle=True, 
          validation_data=(X_train, y_train), 
          callbacks=[earlystopper],
         )

131072
Train on 1962 samples, validate on 1962 samples
Epoch 1/300
1962/1962 [==============================] - 2s 1ms/step - loss: 31.9046 - val_loss: 31.8547
Epoch 2/300
1962/1962 [==============================] - 0s 9us/step - loss: 31.8463 - val_loss: 31.7911
Epoch 3/300
1962/1962 [==============================] - 0s 7us/step - loss: 31.7774 - val_loss: 31.7212
Epoch 4/300
1962/1962 [==============================] - 0s 8us/step - loss: 31.7028 - val_loss: 31.6458
Epoch 5/300
1962/1962 [==============================] - 0s 7us/step - loss: 31.6229 - val_loss: 31.5668
Epoch 6/300
1962/1962 [==============================] - 0s 8us/step - loss: 31.5395 - val_loss: 31.4835
Epoch 7/300
1962/1962 [==============================] - 0s 7us/step - loss: 31.4515 - val_loss: 31.3964
Epoch 8/300
1962/1962 [==============================] - 0s 8us/step - loss: 31.3600 - val_loss: 31.3061
Epoch 9/300
1962/1962 [==============================] - 0s 9us/step - loss: 31.2648 - val_loss: 31.2117


Epoch 78/300
1962/1962 [==============================] - 0s 31us/step - loss: 21.4966 - val_loss: 20.3351
Epoch 79/300
1962/1962 [==============================] - 0s 23us/step - loss: 21.3469 - val_loss: 20.1795
Epoch 80/300
1962/1962 [==============================] - 0s 11us/step - loss: 21.1980 - val_loss: 20.0250
Epoch 81/300
1962/1962 [==============================] - 0s 12us/step - loss: 21.0503 - val_loss: 19.8716
Epoch 82/300
1962/1962 [==============================] - 0s 13us/step - loss: 20.9036 - val_loss: 19.7196
Epoch 83/300
1962/1962 [==============================] - 0s 14us/step - loss: 20.7580 - val_loss: 19.5685
Epoch 84/300
1962/1962 [==============================] - 0s 13us/step - loss: 20.6134 - val_loss: 19.4185
Epoch 85/300
1962/1962 [==============================] - 0s 13us/step - loss: 20.4698 - val_loss: 19.2697
Epoch 86/300
1962/1962 [==============================] - 0s 10us/step - loss: 20.3275 - val_loss: 19.1222
Epoch 87/300
1962/1962 [=============

Epoch 155/300
1962/1962 [==============================] - 0s 10us/step - loss: 12.9873 - val_loss: 11.6009
Epoch 156/300
1962/1962 [==============================] - 0s 10us/step - loss: 12.9076 - val_loss: 11.5261
Epoch 157/300
1962/1962 [==============================] - 0s 12us/step - loss: 12.8284 - val_loss: 11.4521
Epoch 158/300
1962/1962 [==============================] - 0s 13us/step - loss: 12.7496 - val_loss: 11.3789
Epoch 159/300
1962/1962 [==============================] - 0s 11us/step - loss: 12.6713 - val_loss: 11.3068
Epoch 160/300
1962/1962 [==============================] - 0s 9us/step - loss: 12.5935 - val_loss: 11.2355
Epoch 161/300
1962/1962 [==============================] - 0s 38us/step - loss: 12.5161 - val_loss: 11.1651
Epoch 162/300
1962/1962 [==============================] - 0s 15us/step - loss: 12.4393 - val_loss: 11.0950
Epoch 163/300
1962/1962 [==============================] - 0s 16us/step - loss: 12.3629 - val_loss: 11.0254
Epoch 164/300
1962/1962 [====

Epoch 232/300
1962/1962 [==============================] - 0s 11us/step - loss: 8.0562 - val_loss: 7.1236
Epoch 233/300
1962/1962 [==============================] - 0s 8us/step - loss: 8.0053 - val_loss: 7.0793
Epoch 234/300
1962/1962 [==============================] - 0s 11us/step - loss: 7.9546 - val_loss: 7.0355
Epoch 235/300
1962/1962 [==============================] - 0s 38us/step - loss: 7.9042 - val_loss: 6.9922
Epoch 236/300
1962/1962 [==============================] - 0s 17us/step - loss: 7.8539 - val_loss: 6.9493
Epoch 237/300
1962/1962 [==============================] - 0s 10us/step - loss: 7.8039 - val_loss: 6.9068
Epoch 238/300
1962/1962 [==============================] - 0s 10us/step - loss: 7.7542 - val_loss: 6.8647
Epoch 239/300
1962/1962 [==============================] - 0s 12us/step - loss: 7.7048 - val_loss: 6.8219
Epoch 240/300
1962/1962 [==============================] - 0s 8us/step - loss: 7.6557 - val_loss: 6.7792
Epoch 241/300
1962/1962 [=======================

In [19]:
X_pred = model_features.predict(X_train, batch_size=batch_size)\

factors = X_pred[:len(features)]

biases = X_pred[len(features):2*len(features)]

for f, X_p in zip(features, factors):
    for i in range(k_latent):
        X['%s_fm_factor_%d' % (f, i)] = X_p[:,i]

for f, X_p in zip(features, biases):
    X['%s_fm_bias' % (f)] = X_p[:,0]

X.head()

,category_1,category_3,merchant_category_id,num,category_1_fm_factor_0,category_1_fm_factor_1,category_3_fm_factor_0,category_3_fm_factor_1,merchant_category_id_fm_factor_0,merchant_category_id_fm_factor_1,category_1_fm_bias,category_3_fm_bias,merchant_category_id_fm_bias
0,0,0,0,3.555348,0.293158,-0.311619,0.111054,0.18795,0.098653,-0.149103,-0.1195,0.035077,-0.021641
1,0,0,1,5.978886,0.293158,-0.311619,0.111054,0.18795,0.157227,0.203972,-0.1195,0.035077,-0.090545
2,0,0,2,8.223627,0.293158,-0.311619,0.111054,0.18795,0.001411,0.210552,-0.1195,0.035077,0.001771
3,0,0,3,12.132685,0.293158,-0.311619,0.111054,0.18795,-0.112145,0.321727,-0.1195,0.035077,0.061959
4,0,0,4,2.944439,0.293158,-0.311619,0.111054,0.18795,0.189050,0.211982,-0.1195,0.035077,-0.098464


In [20]:
fm_cols= [f for f in X.columns if f not in ['category_1','category_3','merchant_category_id']]
df_historical_trans = df_historical_trans.merge(X,on=['category_1','category_3','merchant_category_id'],how='left')

agg = {}
for col in fm_cols:
    agg[col] = ['mean','max','min','sum']
df_hist = df_historical_trans.groupby(['card_id']).agg(agg)
# change column name
df_hist.columns = pd.Index([e[0] + "_" + e[1] for e in df_hist.columns.tolist()])
df_hist.columns = ['hist_'+ c for c in df_hist.columns]
df_hist.reset_index(inplace=True)
df_hist.head()

,card_id,hist_num_mean,hist_num_max,hist_num_min,hist_num_sum,hist_category_1_fm_factor_0_mean,hist_category_1_fm_factor_0_max,hist_category_1_fm_factor_0_min,hist_category_1_fm_factor_0_sum,hist_category_1_fm_factor_1_mean,...,hist_category_1_fm_bias_min,hist_category_1_fm_bias_sum,hist_category_3_fm_bias_mean,hist_category_3_fm_bias_max,hist_category_3_fm_bias_min,hist_category_3_fm_bias_sum,hist_merchant_category_id_fm_bias_mean,hist_merchant_category_id_fm_bias_max,hist_merchant_category_id_fm_bias_min,hist_merchant_category_id_fm_bias_sum
0,C_ID_00007093c1,12.856863,14.543475,7.347944,1915.672607,0.288862,0.293158,0.270298,43.040459,-0.191934,...,-0.1195,-10.858023,-0.088436,0.122808,-0.128995,-13.176912,0.137758,0.223523,-0.013869,20.525902
1,C_ID_0001238066,12.631703,14.543475,7.510430,1553.699463,0.292786,0.293158,0.270298,36.012718,-0.301263,...,-0.1195,-14.202290,-0.063485,0.122808,-0.128995,-7.808631,0.145267,0.223554,0.015189,17.867800
2,C_ID_0001506ef0,13.159151,15.036073,9.728896,868.503967,0.293158,0.293158,0.293158,19.348431,-0.311619,...,-0.1195,-7.887022,0.032591,0.035077,-0.128995,2.151022,0.164436,0.274356,0.003065,10.852746
3,C_ID_0001793786,12.438538,15.036073,3.496508,2686.724121,0.292946,0.293158,0.270298,63.276421,-0.305722,...,-0.1195,-25.315821,0.031279,0.035077,-0.128995,6.756311,0.136855,0.274356,-0.125758,29.560652
4,C_ID_000183fdda,11.849204,14.543475,7.311218,1706.285400,0.292523,0.293158,0.270298,42.123318,-0.293927,...,-0.1195,-16.215546,-0.064295,0.122808,-0.128995,-9.258501,0.147522,0.274356,-0.024963,21.243233


In [21]:
df_hist.to_csv('./datasets/cate_fm/df_hist_cate_merchantCate_fm_feat.csv',index=False)

In [22]:
df_new = pd.read_csv('./datasets/cate_fm/df_new_cat_merchantCate_fm_feat.csv')
df_temp = pd.merge(df_hist,df_new,on=['card_id'],how='left')
df_temp.head()

,card_id,hist_num_mean,hist_num_max,hist_num_min,hist_num_sum,hist_category_1_fm_factor_0_mean,hist_category_1_fm_factor_0_max,hist_category_1_fm_factor_0_min,hist_category_1_fm_factor_0_sum,hist_category_1_fm_factor_1_mean,...,new_category_1_fm_bias_min,new_category_1_fm_bias_sum,new_category_3_fm_bias_mean,new_category_3_fm_bias_max,new_category_3_fm_bias_min,new_category_3_fm_bias_sum,new_merchant_category_id_fm_bias_mean,new_merchant_category_id_fm_bias_max,new_merchant_category_id_fm_bias_min,new_merchant_category_id_fm_bias_sum
0,C_ID_00007093c1,12.856863,14.543475,7.347944,1915.672607,0.288862,0.293158,0.270298,43.040459,-0.191934,...,-0.067472,-0.134944,-0.085811,-0.085811,-0.085811,-0.171622,0.103764,0.113691,0.093838,0.207529
1,C_ID_0001238066,12.631703,14.543475,7.510430,1553.699463,0.292786,0.293158,0.270298,36.012718,-0.301263,...,-0.067472,-1.466135,-0.059024,0.088308,-0.085811,-1.534612,0.085626,0.115359,0.022911,2.226272
2,C_ID_0001506ef0,13.159151,15.036073,9.728896,868.503967,0.293158,0.293158,0.293158,19.348431,-0.311619,...,-0.067472,-0.134944,-0.012477,-0.012477,-0.012477,-0.024953,0.095970,0.115359,0.076580,0.191940
3,C_ID_0001793786,12.438538,15.036073,3.496508,2686.724121,0.292946,0.293158,0.270298,63.276421,-0.305722,...,-0.067472,-2.091635,-0.012477,-0.012477,-0.012477,-0.386775,0.071884,0.115359,-0.057450,2.228410
4,C_ID_000183fdda,11.849204,14.543475,7.311218,1706.285400,0.292523,0.293158,0.270298,42.123318,-0.293927,...,-0.067472,-0.742193,-0.022495,0.088308,-0.085811,-0.247445,0.076870,0.105201,0.007518,0.845567


In [23]:
df_temp.to_csv('./datasets/cate_fm/df_hist_new_cate_merchantCate_fm_feat.csv',index=False)